In [4]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait, Select
from selenium.webdriver.support import expected_conditions as EC

import requests 
import json
from shutil import which
import time
import os
import pandas as pd
import numpy as np
import re
from bs4 import BeautifulSoup
import urllib.request



In [5]:
def scroll():
    height = driver.execute_script("return document.body.scrollHeight")
    for scrol in range(100,height-300,100):
        driver.execute_script(f"window.scrollTo(0,{scrol})")
        time.sleep(0.1)

In [6]:
def extract_details():
    price = driver.find_element(By.XPATH,"//dd[@class='tm-final-totals']//bdi[1]")
    product_details['price'] = price.text
    sku = driver.find_element(By.XPATH,"//strong[@class='mfc-built-sku']")
    product_details['sku'] = sku.text[8:]
    varient_image = driver.find_element(By.XPATH,"/html[1]/body[1]/div[1]/div[1]/div[1]/main[1]/article[1]/div[1]/div[1]/div[2]/div[1]/figure[1]/a[1]/img[1]")
    product_details['variant_image'] = varient_image.get_attribute('src')
    

In [145]:
chrome_options = Options()

headless = False
if headless:
    chrome_options.add_argument("--headless")
chrome_options.add_argument("--disable-infobars")
chrome_options.add_argument("start-maximized")
chrome_options.add_argument("--disable-extensions")

chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36")

prefs = {"profile.default_content_setting_values.notifications" : 2}
chrome_options.add_experimental_option("prefs",prefs)

# PLease replace your chrome driver path
chrome_path = r"C:\Users\kchai\OneDrive\Documents\Moro\Data scrapping\chromedriver_win32/chromedriver"
global driver
driver = webdriver.Chrome(executable_path=chrome_path, options=chrome_options)
driver.get("https://modernfan.com/ceiling-fans/")

time.sleep(1)

product_url = driver.find_elements(By.XPATH, "//ul[@class = 'products columns-4']/li/a")
items = []
for product in product_url[22:23]:
    print(product.get_attribute('href'))
    product_details = {}
    
    href = product.get_attribute('href')
    product_details['product_url'] = href
    
    

    try:
        name = product.find_element(By.XPATH, ".//h2")
        product_details['name'] = name.text
    except:
        product_details['name'] = None
        
    try:
        hero_image_path = product.find_element(By.XPATH, ".//div/div/img")
        hero_image = hero_image_path.get_attribute('data-lazy-src')

        product_details['hero_image'] = hero_image
    except:
        product_details['hero_image'] = None
        
    try:
        product_description = product.find_element(By.XPATH, ".//div/p")
        product_details['short description'] = product_description.text
    except:
        product_details['short description'] = None

        
    driver.execute_script("window.open('" +href +"');")
    # switch to new window
    driver.switch_to.window(driver.window_handles[1])
    time.sleep(1)
    
    try:
        image = driver.find_elements(By.XPATH, "//div[@class = 'col-xs-4 mfc-image-gallery']/a")
        images = []
        for opt in image:
            images.append(opt.get_attribute('href'))
        product_details['images'] = '\n'.join(images)
    except:
        product_details['images'] = None
        
    try:
        video_img = driver.find_element(By.XPATH, "//a[@class = 'vimeo-video']/img")
        product_details['thumbnail'] = video_img.get_attribute('src')
    except:
        product_details['thumbnail'] = None
        
    try:
        video = driver.find_element(By.XPATH, "//a[@class = 'vimeo-video']")
        product_details['video'] = video.get_attribute('href')
    except:
        product_details['video'] = None
        
    try:
        display_dimensions = driver.find_element(By.XPATH, "//a[@class = 'mfc-image-popup']")
        product_details['display_dimensions'] = display_dimensions.get_attribute('href')
    except:
        product_details['display_dimensions'] = None
        
#     scroll()
    driver.execute_script(f"window.scrollTo(0,800)")
    try:
        features =  driver.find_element(By.LINK_TEXT, "Features")
        features.click()
        time.sleep(1)
        
        pro_desc = driver.find_elements(By.XPATH, "//div[@class = 'woocommerce-tabs wc-tabs-wrapper']/div/ul/li")
        pro_description = []
        for opt in pro_desc:
            pro_description.append(opt.text)
        product_details['product description'] = '\n'.join(pro_description)
    except:
        product_details['product description'] = None
        
    try:
        Performance_Data =  driver.find_element(By.LINK_TEXT, "Performance Data")
        Performance_Data.click()


        tech_description = driver.find_elements(By.XPATH, "//div[@class = 'woocommerce-Tabs-panel woocommerce-Tabs-panel--mfc_performance_data panel entry-content wc-tab']/p")
        product_details['technical description'] = []
        for opt in tech_description:
            product_details['technical description'].append(opt.text)
        product_details['technical description'] = '\n'.join(product_details['technical description'])
    except:
        product_details['product technical description'] = None
        
    try:
        details = driver.find_element(By.XPATH, "(//div[@class = 'woocommerce-Tabs-panel woocommerce-Tabs-panel--mfc_performance_data panel entry-content wc-tab']/p)[2]")
        product_details['details'] = details.text
        try:
            product_details['RPM'] = product_details['details'].split('\n')[0].split(':')[-1]
        except:
            product_details['RPM'] = None
        try:
            product_details['CFM'] = product_details['details'].split('\n')[1].split(':')[-1]
        except:
            product_details['CFM'] = None
            
        try:
            product_details['Watts (fan only)'] = product_details['details'].split('\n')[2].split(':')[-1]
        except:
            product_details['Watts (fan only)'] = None
            
        try:
            product_details['CFM/Watt'] = product_details['details'].split('\n')[3].split(':')[-1]
        except:
            product_details['CFM/Watt'] = None
        
    except:
        product_details['details'] = None
        
    try:
        energy = driver.find_element(By.XPATH, "(//div[@class = 'woocommerce-Tabs-panel woocommerce-Tabs-panel--mfc_performance_data panel entry-content wc-tab']/p)[4]")
        product_details['energy'] = details.text
        try:
            product_details['Average Energy Use'] = product_details['energy'].split('\n')[0].split(':')[-1]
        except:
            product_details['Average Energy Use'] = None
        try:
            product_details['Average Airflow'] = product_details['energy'].split('\n')[1].split(':')[-1]
        except:
            product_details['Average Airflow'] = None
            
        try:
            product_details['Yearly Energy Cost'] = product_details['energy'].split('\n')[2].split(':')[-1]
        except:
            product_details['Yearly Energy Cost'] = None
            
        try:
            product_details['Average Efficiency'] = product_details['energy'].split('\n')[3].split(':')[-1]
        except:
            product_details['Average Efficiency'] = None
    except:
        product_details['energy'] = None

    try:
        Spec_Sheet = driver.find_element(By.XPATH, "//div[@class = 'pull-right mfc-tab-buttons']/a[1]")
        product_details['Spec Sheet'] = Spec_Sheet.get_attribute('href')
    except:
        product_details['Spec Sheet'] = None
        
    try:
        Instructions = driver.find_element(By.XPATH, "//div[@class = 'pull-right mfc-tab-buttons']/a[2]")
        product_details['Instructions'] = Instructions.get_attribute('href')
    except:
        product_details['Instructions'] = None
        
        
    try:
        product_details['files'] ='Spec Sheet:' + product_details['Spec Sheet'] + '\n' + 'Instructions: ' + product_details['Instructions']
    except:
        product_details['files'] = None
        
    product_details['options'] = []
    
    try:
        options_desc = driver.find_elements(By.XPATH, "//div[@class = 'tc-element-inner-wrap']/div/label")
        options = []
        for opt in options_desc:
            options.append(opt.text)
        product_details['options'] = options
    except:
        product_details['options'] = None
    time.sleep(1)  
    
    try:
        fan_finish = driver.find_elements(By.XPATH, "/html[1]/body[1]/div[1]/div[1]/div[1]/main[1]/article[1]/div[1]/div[1]/div[2]/div[2]/form[1]/div[1]/ul[1]/li[1]/div[1]/div[1]/div[1]/div[1]/div[1]/div[1]/div[1]/div[1]/div[1]/ul[1]/li/label/span/img")
        options_fan_finish = []
        for opt in fan_finish:
            options_fan_finish.append(opt.get_attribute('alt'))
        #         print(options_fan_finish)
        product_details[f'option({options[0]})'] = options_fan_finish
    except:
        product_details[f'option({options[0]})'] = None
        
    try:
        blade_color = driver.find_elements(By.XPATH, "/html[1]/body[1]/div[1]/div[1]/div[1]/main[1]/article[1]/div[1]/div[1]/div[2]/div[2]/form[1]/div[1]/ul[1]/li[1]/div[1]/div[1]/div[1]/div[1]/div[1]/div[2]/div/div/div/ul/li/label/span/img")
        options_blade_color = []
        for opt in blade_color:
            options_blade_color.append(opt.get_attribute('alt'))
        #         print(options_fan_finish)
        product_details[f'option({options[1]})'] = options_blade_color
    except:
        print('2nd option doesnt exist')
        
    try:
        light = driver.find_elements(By.XPATH, "/html[1]/body[1]/div[1]/div[1]/div[1]/main[1]/article[1]/div[1]/div[1]/div[2]/div[2]/form[1]/div[1]/ul[1]/li[1]/div[1]/div[1]/div[1]/div[1]/div[1]/div[3]/div/div/div/ul/li/label/span/img")
        options_light = []
        for opt in light:
            options_light.append(opt.get_attribute('alt'))
        #         print(options_fan_finish)
        product_details[f'option({options[2]})'] = options_light
    except:
        print('3rd option doesnt exist')
        



            
    fan_finish = driver.find_elements(By.XPATH, "/html[1]/body[1]/div[1]/div[1]/div[1]/main[1]/article[1]/div[1]/div[1]/div[2]/div[2]/form[1]/div[1]/ul[1]/li[1]/div[1]/div[1]/div[1]/div[1]/div[1]/div[1]/div/div/div/ul/li/label/span")
    blade_color = driver.find_elements(By.XPATH, "/html[1]/body[1]/div[1]/div[1]/div[1]/main[1]/article[1]/div[1]/div[1]/div[2]/div[2]/form[1]/div[1]/ul[1]/li[1]/div[1]/div[1]/div[1]/div[1]/div[1]/div[2]/div/div/div/ul/li/label/span")
    light = driver.find_elements(By.XPATH, "/html[1]/body[1]/div[1]/div[1]/div[1]/main[1]/article[1]/div[1]/div[1]/div[2]/div[2]/form[1]/div[1]/ul[1]/li[1]/div[1]/div[1]/div[1]/div[1]/div[1]/div[3]/div/div/div/ul/li/label/span")

    
    if len(options_fan_finish) >0:
        for finish in fan_finish:
            webdriver.ActionChains(driver).move_to_element(finish ).click(finish ).perform()
            finish_name = finish.find_element(By.XPATH, ".//img").get_attribute('alt')
            finish_swatch = finish.find_element(By.XPATH, ".//img").get_attribute('src')
            product_details[f'option({options[0][10:]})'] = finish_name + ':' +finish_swatch
            if len(options_blade_color) >0:
                for blade in blade_color:
                    blade.click()
                    blade_name = blade.find_element(By.XPATH, ".//img").get_attribute('alt')
                    blade_swatch = blade.find_element(By.XPATH, ".//img").get_attribute('src')
                    
                    product_details[f'option({options[1][10:]})'] = blade_name + ':' + blade_swatch
                    if len(options_light) >0:
                        for lig in light:
                            try:
                                lig.click()
                            except:
                                pass
                            light_name = lig.find_element(By.XPATH, ".//img").get_attribute('alt')
                            light_swatch = lig.find_element(By.XPATH, ".//img").get_attribute('src')
                            product_details[f'option({options[2][10:]})'] = light_name + ':' + light_swatch
                            try:
                                control = driver.find_elements(By.XPATH, "/html[1]/body[1]/div[1]/div[1]/div[1]/main[1]/article[1]/div[1]/div[1]/div[2]/div[2]/form[1]/div[1]/ul[1]/li[1]/div[1]/div[1]/div[1]/div[1]/div[1]/div[4]/div/div/div/ul/li/label/span/img")
                                options_control = []
                                for opt in control:
                                    options_control.append(opt.get_attribute('alt'))
                                product_details['option(remote_control)'] = options_control
                            except:
                                print('4th option doesnt exist')
                            control = driver.find_elements(By.XPATH, "/html[1]/body[1]/div[1]/div[1]/div[1]/main[1]/article[1]/div[1]/div[1]/div[2]/div[2]/form[1]/div[1]/ul[1]/li[1]/div[1]/div[1]/div[1]/div[1]/div[1]/div[4]/div/div/div/ul/li/label/span")
                            control2 = driver.find_elements(By.XPATH, "/html[1]/body[1]/div[1]/div[1]/div[1]/main[1]/article[1]/div[1]/div[1]/div[2]/div[2]/form[1]/div[1]/ul[1]/li[1]/div[1]/div[1]/div[1]/div[1]/div[1]/div[5]/div/div/div/ul/li/label/span")

                            
                            if len(options_control) >0:
                                try:
                                    for ctrl in control:
                                        ctrl.click()
                                        extract_details()
                                        control_name =  ctrl.find_element(By.XPATH, ".//img").get_attribute('alt')
                                        control_swatch =  ctrl.find_element(By.XPATH, ".//img").get_attribute('src')
                                        
                                        product_details['option(control)'] =control_name + ':' + control_swatch
                                        items.append(product_details.copy())
                                except:
                                    for ctrl in control2:
                                        try:
                                            ctrl.click()
                                        except:
                                            pass
                                        extract_details()
                                        control_name =  ctrl.find_element(By.XPATH, ".//img").get_attribute('alt')
                                        control_swatch =  ctrl.find_element(By.XPATH, ".//img").get_attribute('src')
                                        
                                        product_details['option(control)'] =control_name + ':' + control_swatch
                                        items.append(product_details.copy())
                            else:
                                extract_details()
                                items.append(product_details.copy())
                    else:
                        extract_details()
                        items.append(product_details.copy())
            else:
                extract_details()
                items.append(product_details.copy())

    
    
    
    driver.close()
    #back to main window
    driver.switch_to.window(driver.window_handles[0])
        
    items.append(product_details.copy())
    
    

C:\Users\kchai\AppData\Local\Temp\ipykernel_9936\1625451819.py:18: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=chrome_path, options=chrome_options)


https://modernfan.com/ceiling-fans/gusto-flush-56/


In [13]:
'Item #: ARB-DB-54-EB-NL-WC'[8:]

'ARB-DB-54-EB-NL-WC'

In [146]:
len(items)

8

In [24]:
len(items)

2600

In [147]:
import pandas as pd
import numpy as np

df = pd.DataFrame(items, )
df.head()


,product_url,name,hero_image,short description,images,thumbnail,video,display_dimensions,product description,technical description,...,option(2. Choose Blade Color),option(3. Choose Light Option),option(Fan Finish),option(Blade Color),option(Light Option),option(remote_control),price,sku,variant_image,option(control)
0,https://modernfan.com/ceiling-fans/gusto-flush...,Gusto Flush | 56″,https://modernfan.com/wp-content/uploads/mfc-p...,Bright Nickel Finish Only\nOptional LED\nLimit...,https://modernfan.com/wp-content/uploads/2021/...,https://i.vimeocdn.com/video/801122621-8b79a2b...,http://vimeo.com/349988528,https://modernfan.com/wp-content/uploads/2021/...,153mm x 22mm AC motor with reverse switch on f...,High Speed Performance Data:\nBased on DOE sta...,...,[Nickel],"[No Light, 20W LED- #173 (+$80.00)]",Bright Nickel:https://modernfan.com/wp-content...,Nickel:https://modernfan.com/wp-content/upload...,No Light:https://modernfan.com/wp-content/uplo...,"[Fan & Light – 3 Wire (+$25.00), Handheld Remo...",$440.00,GUS-FM-BN-56-NK-NL-001,https://modernfan.com/wp-content/uploads/mfc-p...,Fan Speed Only (+$20.00):https://modernfan.com...
1,https://modernfan.com/ceiling-fans/gusto-flush...,Gusto Flush | 56″,https://modernfan.com/wp-content/uploads/mfc-p...,Bright Nickel Finish Only\nOptional LED\nLimit...,https://modernfan.com/wp-content/uploads/2021/...,https://i.vimeocdn.com/video/801122621-8b79a2b...,http://vimeo.com/349988528,https://modernfan.com/wp-content/uploads/2021/...,153mm x 22mm AC motor with reverse switch on f...,High Speed Performance Data:\nBased on DOE sta...,...,[Nickel],"[No Light, 20W LED- #173 (+$80.00)]",Bright Nickel:https://modernfan.com/wp-content...,Nickel:https://modernfan.com/wp-content/upload...,No Light:https://modernfan.com/wp-content/uplo...,"[Fan & Light – 3 Wire (+$25.00), Handheld Remo...",$465.00,GUS-FM-BN-56-NK-NL-003,https://modernfan.com/wp-content/uploads/mfc-p...,Handheld Remote (+$45.00):https://modernfan.co...
2,https://modernfan.com/ceiling-fans/gusto-flush...,Gusto Flush | 56″,https://modernfan.com/wp-content/uploads/mfc-p...,Bright Nickel Finish Only\nOptional LED\nLimit...,https://modernfan.com/wp-content/uploads/2021/...,https://i.vimeocdn.com/video/801122621-8b79a2b...,http://vimeo.com/349988528,https://modernfan.com/wp-content/uploads/2021/...,153mm x 22mm AC motor with reverse switch on f...,High Speed Performance Data:\nBased on DOE sta...,...,[Nickel],"[No Light, 20W LED- #173 (+$80.00)]",Bright Nickel:https://modernfan.com/wp-content...,Nickel:https://modernfan.com/wp-content/upload...,No Light:https://modernfan.com/wp-content/uplo...,"[Fan & Light – 3 Wire (+$25.00), Handheld Remo...",$485.00,GUS-FM-BN-56-NK-NL-005,https://modernfan.com/wp-content/uploads/mfc-p...,Wall & Remote Combo (+$65.00):https://modernfa...
3,https://modernfan.com/ceiling-fans/gusto-flush...,Gusto Flush | 56″,https://modernfan.com/wp-content/uploads/mfc-p...,Bright Nickel Finish Only\nOptional LED\nLimit...,https://modernfan.com/wp-content/uploads/2021/...,https://i.vimeocdn.com/video/801122621-8b79a2b...,http://vimeo.com/349988528,https://modernfan.com/wp-content/uploads/2021/...,153mm x 22mm AC motor with reverse switch on f...,High Speed Performance Data:\nBased on DOE sta...,...,[Nickel],"[No Light, 20W LED- #173 (+$80.00)]",Bright Nickel:https://modernfan.com/wp-content...,Nickel:https://modernfan.com/wp-content/upload...,20W LED- #173 (+$80.00):https://modernfan.com/...,"[Fan & Light – 3 Wire (+$25.00), Handheld Remo...",$525.00,GUS-FM-BN-56-NK-173-002,https://modernfan.com/wp-content/uploads/mfc-p...,Fan & Light – 3 Wire (+$25.00):https://modernf...
4,https://modernfan.com/ceiling-fans/gusto-flush...,Gusto Flush | 56″,https://modernfan.com/wp-content/uploads/mfc-p...,Bright Nickel Finish Only\nOptional LED\nLimit...,https://modernfan.com/wp-content/uploads/2021/...,https://i.vimeocdn.com/video/801122621-8b79a2b...,http://vimeo.com/349988528,https://modernfan.com/wp-content/uploads/2021/...,153mm x 2

In [148]:
df.to_excel("modernfan_gusto.xlsx")

In [131]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29 entries, 0 to 28
Data columns (total 35 columns):
 #   Column                          Non-Null Count  Dtype 
---  ------                          --------------  ----- 
 0   product_url                     29 non-null     object
 1   name                            29 non-null     object
 2   hero_image                      29 non-null     object
 3   short description               29 non-null     object
 4   images                          29 non-null     object
 5   thumbnail                       29 non-null     object
 6   video                           29 non-null     object
 7   display_dimensions              29 non-null     object
 8   product description             29 non-null     object
 9   technical description           29 non-null     object
 10  details                         29 non-null     object
 11  RPM                             29 non-null     object
 12  CFM                             29 non-null     obje

In [117]:
df['option(control)']

0     Fan Speed Only (+$20.00):https://modernfan.com...
1     Handheld Remote (+$45.00):https://modernfan.co...
2     Wall & Remote Combo (+$65.00):https://modernfa...
3     Fan Speed Only (+$20.00):https://modernfan.com...
4     Handheld Remote (+$45.00):https://modernfan.co...
5     Wall & Remote Combo (+$65.00):https://modernfa...
6     Fan Speed Only (+$20.00):https://modernfan.com...
7     Handheld Remote (+$45.00):https://modernfan.co...
8     Wall & Remote Combo (+$65.00):https://modernfa...
9     Fan Speed Only (+$20.00):https://modernfan.com...
10    Handheld Remote (+$45.00):https://modernfan.co...
11    Wall & Remote Combo (+$65.00):https://modernfa...
12    Fan Speed Only (+$20.00):https://modernfan.com...
13    Handheld Remote (+$45.00):https://modernfan.co...
14    Wall & Remote Combo (+$65.00):https://modernfa...
15    Fan Speed Only (+$20.00):https://modernfan.com...
16    Handheld Remote (+$45.00):https://modernfan.co...
17    Wall & Remote Combo (+$65.00):https://mode

In [118]:
df1 = df.drop(['thumbnail', 'option(1. Choose Finish)', 'option(2. Choose Length)', 'option(3. Chose Light Option)', 'option(3. Choose Control)', 'option(2. Chose Blade Color)',\
               'option(3. Choose Light Option)', 'option(2. Chose Blade Color)', 'option(2. Choose Blade Color)', 'option(1. Choose Fan Finish)'\
              , 'option()', 'options', 'details', 'energy','option(remote_control)', 'Instructions', 'Spec Sheet' ], axis=1)
# df1


KeyError: "['option(1. Choose Finish)', 'option(2. Choose Length)', 'option(3. Chose Light Option)', 'option(3. Choose Control)', 'option(2. Chose Blade Color)', 'option(3. Choose Light Option)', 'option(2. Chose Blade Color)'] not found in axis"

In [119]:
df1.info()

NameError: name 'df1' is not defined

In [34]:
df1 = df1.replace({np.nan: None})

In [35]:
df1['option(blade Color)'] = None
df1['option(light Option)'] = None
for i in range(len(df1)):
    if (df1['option(Blade Color)'][i] is None) and (df1['option(lade Color)'][i] is not None):
        df1['option(blade Color)'][i] = df1['option(lade Color)'][i]
    elif (df1['option(Blade Color)'][i] is not None) and (df1['option(lade Color)'][i] is None):
        df1['option(blade Color)'][i] = df1['option(Blade Color)'][i]
    else:
        df1['option(blade Color)'][i] = None
        
    if (df1['option(Light Option)'][i] is None) and (df1['option(ight Option)'][i] is not None):
        df1['option(light Option)'][i] = df1['option(ight Option)'][i]
    elif (df1['option(Light Option)'][i] is not None) and (df1['option(ight Option)'][i] is None):
        df1['option(light Option)'][i] = df1['option(Light Option)'][i]
    else:
        df1['option(light Option)'][i] = None

In [38]:
df1.drop(['option(Blade Color)', 'option(lade Color)', 'option(Light Option)', 'option(ight Option)'], axis = 1, inplace = True)

In [36]:
df1['finish'] = df1['option(Finish)']

In [41]:
df1.hero_image[1]

'https://modernfan.com/wp-content/uploads/mfc-product-images-600/ARB-GT-54-MP-NL.jpg'

In [184]:
df1.reset_index(inplace=True)


In [186]:
df1 = df1.loc[:,~df1.columns.duplicated()]

In [44]:
df1.columns.tolist()

['product_url',
 'name',
 'hero_image',
 'short description',
 'images',
 'video',
 'display_dimensions',
 'product description',
 'technical description',
 'RPM',
 'CFM',
 'Watts (fan only)',
 'CFM/Watt',
 'Average Energy Use',
 'Average Airflow',
 'Yearly Energy Cost',
 'Average Efficiency',
 'files',
 'option(Fan Finish)',
 'price',
 'sku',
 'variant_image',
 'option(control)',
 'product technical description',
 'option(Finish)',
 'option(Length)',
 'option(blade Color)',
 'option(light Option)',
 'finish']

In [43]:
df_full = df1.reindex(columns=[*df1.columns.tolist(), 'GTIN', 'option(size)', 'option(cage)', \
                             'option(color)' ,'option(canopy color)', 'color',\
                                'depth', 'width', 'height', 'depth', 'weight' \
                    'diameter', 'arm height', 'seat height', 'table clearance height', \
                    'maximum dimensions','made_in', 'canopy dimensions', 'installations', 'designer',\
                              'collection', 'categories', 'available_new' , 'voltage', 'bulb_type', 'bulb_text', 'UL Listed', \
                    'certifications', 'warranty & care', 'position', 'notes'], fill_value= None)

In [190]:
df_full.drop(['level_0', 'index'], axis = 1, inplace = True)

In [45]:
df_full.to_excel('modernfan.xlsx')

In [121]:
df = pd.read_excel(r"C:\Users\kchai\Downloads\moderfan.xlsx")
df

,name,price,sku,product_url,hero_image,short description,images,variant_image,option(Fan Finish),option(Light Option),...,categories,available_new,voltage,bulb_type,bulb_text,UL Listed,certifications,warranty & care,position,notes
0,Arbor DC | 54″,$615.00,ARB-DB-54-EB-NL-WC,https://modernfan.com/ceiling-fans/arbor-dc-54/,https://modernfan.com/wp-content/uploads/mfc-p...,Three Fan Finishes\nSolid Wood Blades in Five ...,https://modernfan.com/wp-content/uploads/2020/...,https://modernfan.com/wp-content/uploads/mfc-p...,Dark Bronze: https://modernfan.com/wp-content/...,No Light: https://modernfan.com/wp-content/upl...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Arbor DC | 54″,$615.00,ARB-DB-54-EB-NL-RC,https://modernfan.com/ceiling-fans/arbor-dc-54/,https://modernfan.com/wp-content/uploads/mfc-p...,Three Fan Finishes\nSolid Wood Blades in Five ...,https://modernfan.com/wp-content/uploads/2020/...,https://modernfan.com/wp-content/uploads/mfc-p...,Dark Bronze: https://modernfan.com/wp-content/...,No Light: https://modernfan.com/wp-content/upl...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Arbor DC | 54″,$640.00,ARB-DB-54-EB-NL-CC,https://modernfan.com/ceiling-fans/arbor-dc-54/,https://modernfan.com/wp-content/uploads/mfc-p...,Three Fan Finishes\nSolid Wood Blades in Five ...,https://modernfan.com/wp-content/uploads/2020/...,https://modernfan.com/wp-content/uploads/mfc-p...,Dark Bronze: https://modernfan.com/wp-content/...,No Light: https://modernfan.com/wp-content/upl...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Arbor DC | 54″,$680.00,ARB-DB-54-EB-870-WC,https://modernfan.com/ceiling-fans/arbor-dc-54/,https://modernfan.com/wp-content/uploads/mfc-p...,Three Fan Finishes\nSolid Wood Blades in Five ...,https://modernfan.com/wp-content/uploads/2020/...,https://modernfan.com/wp-content/uploads/mfc-p...,Dark Bronze: https://modernfan.com/wp-content/...,17W LED- #870 (+$65.00): https://modernfan.com...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Arbor DC | 54″,$680.00,ARB-DB-54-EB-870-RC,https://modernfan.com/ceiling-fans/arbor-dc-54/,https://modernfan.com/wp-content/uploads/mfc-p...,Three Fan Finishes\nSolid Wood Blades in Five ...,https://modernfan.com/wp-content/uploads/2020/...,https://modernfan.com/wp-content/uploads/mfc-p...,Dark Bronze: https://modernfan.com/wp-content/...,17W LED- #870 (+$65.00): https://modernfan.com...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2485,45 Degree Slope Kit,$25.00,045-GV,https://modernfan.com/ceiling-fans/45-degree-s...,https://modernfan.com/wp-content/uploads/mfc-p...,Eight finishes\nFor slopes exceeding 33 degree...,https://modernfan.com/wp-content/uploads/mfc-p...,https://modernfan.com/wp-content/uploads/mfc-p...,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2486,45 Degree Slope Kit,$25.00,045-GW,https://modernfan.com/ceiling-fans/45-degree-s...,https://modernfan.com/wp-content/uploads/mfc-p...,Eight finishes\nFor slopes exceeding 33 degree...,https://modernfan.com/wp-content/uploads/mfc-p...,https://modernfan.com/wp-content/uploads/mfc-p...,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2487,45 Degree Slope Kit,$25.00,045-GT,https://modernfan.com/ceiling-fans/45-degree-s...,https://modernfan.com/wp-content/uploads/mfc-p...,Eight finishes\nFor slopes exceeding 33 degree...,https://modernfan.com/wp-content/uploads/mfc-p...,https://modernfan.com/wp-content/uploads/mfc-p...,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2488,45 Degree Slope Kit,$25.00,045-MN,https://modernfan.com/ceiling-fans/45-degree-s...,https://modernfan.com/wp-content/uploads/mfc-p...,Eight finishes\nFor slopes exceeding 33 degree...,https://modernfan.com/wp-content/uploads/mfc-p...,https://modernfan.com/wp-content/uploads/mfc-p...,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [123]:
df = df.drop_duplicates(keep='first' , ignore_index=True)
df

,name,price,sku,product_url,hero_image,short description,images,variant_image,option(Fan Finish),option(Light Option),...,categories,available_new,voltage,bulb_type,bulb_text,UL Listed,certifications,warranty & care,position,notes
0,Arbor DC | 54″,$615.00,ARB-DB-54-EB-NL-WC,https://modernfan.com/ceiling-fans/arbor-dc-54/,https://modernfan.com/wp-content/uploads/mfc-p...,Three Fan Finishes\nSolid Wood Blades in Five ...,https://modernfan.com/wp-content/uploads/2020/...,https://modernfan.com/wp-content/uploads/mfc-p...,Dark Bronze: https://modernfan.com/wp-content/...,No Light: https://modernfan.com/wp-content/upl...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Arbor DC | 54″,$615.00,ARB-DB-54-EB-NL-RC,https://modernfan.com/ceiling-fans/arbor-dc-54/,https://modernfan.com/wp-content/uploads/mfc-p...,Three Fan Finishes\nSolid Wood Blades in Five ...,https://modernfan.com/wp-content/uploads/2020/...,https://modernfan.com/wp-content/uploads/mfc-p...,Dark Bronze: https://modernfan.com/wp-content/...,No Light: https://modernfan.com/wp-content/upl...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Arbor DC | 54″,$640.00,ARB-DB-54-EB-NL-CC,https://modernfan.com/ceiling-fans/arbor-dc-54/,https://modernfan.com/wp-content/uploads/mfc-p...,Three Fan Finishes\nSolid Wood Blades in Five ...,https://modernfan.com/wp-content/uploads/2020/...,https://modernfan.com/wp-content/uploads/mfc-p...,Dark Bronze: https://modernfan.com/wp-content/...,No Light: https://modernfan.com/wp-content/upl...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Arbor DC | 54″,$680.00,ARB-DB-54-EB-870-WC,https://modernfan.com/ceiling-fans/arbor-dc-54/,https://modernfan.com/wp-content/uploads/mfc-p...,Three Fan Finishes\nSolid Wood Blades in Five ...,https://modernfan.com/wp-content/uploads/2020/...,https://modernfan.com/wp-content/uploads/mfc-p...,Dark Bronze: https://modernfan.com/wp-content/...,17W LED- #870 (+$65.00): https://modernfan.com...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Arbor DC | 54″,$680.00,ARB-DB-54-EB-870-RC,https://modernfan.com/ceiling-fans/arbor-dc-54/,https://modernfan.com/wp-content/uploads/mfc-p...,Three Fan Finishes\nSolid Wood Blades in Five ...,https://modernfan.com/wp-content/uploads/2020/...,https://modernfan.com/wp-content/uploads/mfc-p...,Dark Bronze: https://modernfan.com/wp-content/...,17W LED- #870 (+$65.00): https://modernfan.com...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2485,45 Degree Slope Kit,$25.00,045-GV,https://modernfan.com/ceiling-fans/45-degree-s...,https://modernfan.com/wp-content/uploads/mfc-p...,Eight finishes\nFor slopes exceeding 33 degree...,https://modernfan.com/wp-content/uploads/mfc-p...,https://modernfan.com/wp-content/uploads/mfc-p...,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2486,45 Degree Slope Kit,$25.00,045-GW,https://modernfan.com/ceiling-fans/45-degree-s...,https://modernfan.com/wp-content/uploads/mfc-p...,Eight finishes\nFor slopes exceeding 33 degree...,https://modernfan.com/wp-content/uploads/mfc-p...,https://modernfan.com/wp-content/uploads/mfc-p...,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2487,45 Degree Slope Kit,$25.00,045-GT,https://modernfan.com/ceiling-fans/45-degree-s...,https://modernfan.com/wp-content/uploads/mfc-p...,Eight finishes\nFor slopes exceeding 33 degree...,https://modernfan.com/wp-content/uploads/mfc-p...,https://modernfan.com/wp-content/uploads/mfc-p...,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2488,45 Degree Slope Kit,$25.00,045-MN,https://modernfan.com/ceiling-fans/45-degree-s...,https://modernfan.com/wp-content/uploads/mfc-p...,Eight finishes\nFor slopes exceeding 33 degree...,https://modernfan.com/wp-content/uploads/mfc-p...,https://modernfan.com/wp-content/uploads/mfc-p...,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [122]:
df[df[]]

SyntaxError: invalid syntax (2896893402.py, line 1)

In [124]:
df_clr = df[df['name'] != 'Solus | 52″'][opt_columns]
df_clr

,option(Fan Finish),option(Light Option),option(control),option(Finish),option(Length),option(blade Color),name,sku
0,Dark Bronze: https://modernfan.com/wp-content/...,No Light: https://modernfan.com/wp-content/upl...,Wall Control: https://modernfan.com/wp-content...,NaN,NaN,Ebony: https://modernfan.com/wp-content/upload...,Arbor DC | 54″,ARB-DB-54-EB-NL-WC
1,Dark Bronze: https://modernfan.com/wp-content/...,No Light: https://modernfan.com/wp-content/upl...,Remote Control: https://modernfan.com/wp-conte...,NaN,NaN,Ebony: https://modernfan.com/wp-content/upload...,Arbor DC | 54″,ARB-DB-54-EB-NL-RC
2,Dark Bronze: https://modernfan.com/wp-content/...,No Light: https://modernfan.com/wp-content/upl...,Wall/Remote Combo (+$25.00): https://modernfan...,NaN,NaN,Ebony: https://modernfan.com/wp-content/upload...,Arbor DC | 54″,ARB-DB-54-EB-NL-CC
3,Dark Bronze: https://modernfan.com/wp-content/...,17W LED- #870 (+$65.00): https://modernfan.com...,Wall Control: https://modernfan.com/wp-content...,NaN,NaN,Ebony: https://modernfan.com/wp-content/upload...,Arbor DC | 54″,ARB-DB-54-EB-870-WC
4,Dark Bronze: https://modernfan.com/wp-content/...,17W LED- #870 (+$65.00): https://modernfan.com...,Remote Control: https://modernfan.com/wp-conte...,NaN,NaN,Ebony: https://modernfan.com/wp-content/upload...,Arbor DC | 54″,ARB-DB-54-EB-870-RC
...,...,...,...,...,...,...,...,...
2485,NaN,NaN,NaN,Galvanized: https://modernfan.com/wp-content/u...,NaN,NaN,45 Degree Slope Kit,045-GV
2486,NaN,NaN,NaN,Gloss White: https://modernfan.com/wp-content/...,NaN,NaN,45 Degree Slope Kit,045-GW
2487,NaN,NaN,NaN,Graphite: https://modernfan.com/wp-content/upl...,NaN,NaN,45 Degree Slope Kit,045-GT
2488,NaN,NaN,NaN,Matte Nickel: https://modernfan.com/wp-content...,NaN,NaN,45 Degree Slope Kit,045-MN


In [ ]:
df_clr.to_excel('modern')

In [82]:
for i in range(len(df)):
    if df['images'][i] is None:
        temp_imgs = list(df[df['name'] == df['name'][i]]['variant_image'].unique())
        df['images'][i] = '\n'.join(temp_imgs)
    else:
        try:
            temp_imgs = df['images'][i].split('\n') + list(df[df['name'] == df['name'][i]]['variant_image'].unique())
            df['images'][i] = '\n'.join(temp_imgs)
        except:
            try:
                df['images'][i] = '\n'.join(list(df[df['name'] == df['name'][i]]['variant_image'].unique()))
            except:
                df['images'][i] = df['images'][i]
                print('check it')
    print(i)

C:\Users\kchai\AppData\Local\Temp\ipykernel_9936\3487842423.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['images'][i] = '\n'.join(temp_imgs)


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

1929
1930
1931
1932
1933
1934
1935
1936
1937
1938
1939
1940
1941
1942
1943
1944
1945
1946
1947
1948
1949
1950
1951
1952
1953
1954
1955
1956
1957
1958
1959
1960
1961
1962
1963
1964
1965
1966
1967
1968
1969
1970
1971
1972
1973
1974
1975
1976
1977
1978
1979
1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
2024
2025
2026
2027
2028
2029
2030
2031
2032
2033
2034
2035
2036
2037
2038
2039
2040
2041
2042
2043
2044
2045
2046
2047
2048
2049
2050
2051
2052
2053
2054
2055
2056
2057
2058
2059
2060
2061
2062
2063
2064
2065
2066
2067
2068
2069
2070
2071
2072
2073
2074
2075
2076
2077
2078
2079
2080
2081
2082
2083
2084
2085
2086
2087
2088
2089
2090
2091
2092
2093
2094
2095
2096
2097
2098
2099
2100
2101
2102
2103
2104
2105
2106
2107
2108
2109
2110
2111
2112
2113
2114
2115
2116
2117
2118
2119
2120
2121
2122
2123
2124
2125
2126
2127
2128


C:\Users\kchai\AppData\Local\Temp\ipykernel_9936\3487842423.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['images'][i] = '\n'.join(list(df[df['name'] == df['name'][i]]['variant_image'].unique()))


In [84]:
df.iloc[2155]

Unnamed: 0                                                                    2155
index                                                                         2155
product_url                      https://modernfan.com/ceiling-fans/velo-flush-...
name                                                           Velo Flush DC | 48″
hero_image                       https://modernfan.com/wp-content/uploads/mfc-p...
short description                Three Fan Finishes\nEight Blade Colors\nOption...
images                           https://modernfan.com/wp-content/uploads/2021/...
video                                                   http://vimeo.com/349984150
display_dimensions               https://modernfan.com/wp-content/uploads/2021/...
product description              6-speed, reversible DC motor\nDie cast aluminu...
technical description            High Speed Performance Data:\nBased on DOE sta...
RPM                                                                          235.0
CFM 

In [ ]:
df = pd.read_excel('  ')

In [91]:
opt_columns = [col for col in df.columns if 'option' in col]
opt_columns

['option(Fan Finish)',
 'option(Light Option)',
 'option(control)',
 'option(Finish)',
 'option(Length)',
 'option(blade Color)']

In [93]:
opt_columns.extend(['name', 'sku'])
opt_columns

['option(Fan Finish)',
 'option(Light Option)',
 'option(control)',
 'option(Finish)',
 'option(Length)',
 'option(blade Color)',
 'name',
 'sku']

In [94]:
df = df.drop_duplicates(subset = opt_columns, keep='first' , ignore_index=True)
df

,Unnamed: 0,index,product_url,name,hero_image,short description,images,video,display_dimensions,product description,...,categories,available_new,voltage,bulb_type,bulb_text,UL Listed,certifications,warranty & care,position,notes
0,0,0,https://modernfan.com/ceiling-fans/arbor-dc-54/,Arbor DC | 54″,https://modernfan.com/wp-content/uploads/mfc-p...,Three Fan Finishes\nSolid Wood Blades in Five ...,https://modernfan.com/wp-content/uploads/2020/...,http://vimeo.com/437221607,https://modernfan.com/wp-content/uploads/2021/...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,1,https://modernfan.com/ceiling-fans/arbor-dc-54/,Arbor DC | 54″,https://modernfan.com/wp-content/uploads/mfc-p...,Three Fan Finishes\nSolid Wood Blades in Five ...,https://modernfan.com/wp-content/uploads/2020/...,http://vimeo.com/437221607,https://modernfan.com/wp-content/uploads/2021/...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,2,https://modernfan.com/ceiling-fans/arbor-dc-54/,Arbor DC | 54″,https://modernfan.com/wp-content/uploads/mfc-p...,Three Fan Finishes\nSolid Wood Blades in Five ...,https://modernfan.com/wp-content/uploads/2020/...,http://vimeo.com/437221607,https://modernfan.com/wp-content/uploads/2021/...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,3,https://modernfan.com/ceiling-fans/arbor-dc-54/,Arbor DC | 54″,https://modernfan.com/wp-content/uploads/mfc-p...,Three Fan Finishes\nSolid Wood Blades in Five ...,https://modernfan.com/wp-content/uploads/2020/...,http://vimeo.com/437221607,https://modernfan.com/wp-content/uploads/2021/...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,4,https://modernfan.com/ceiling-fans/arbor-dc-54/,Arbor DC | 54″,https://modernfan.com/wp-content/uploads/mfc-p...,Three Fan Finishes\nSolid Wood Blades in Five ...,https://modernfan.com/wp-content/uploads/2020/...,http://vimeo.com/437221607,https://modernfan.com/wp-content/uploads/2021/...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2485,2518,2518,https://modernfan.com/ceiling-fans/45-degree-s...,45 Degree Slope Kit,https://modernfan.com/wp-content/uploads/mfc-p...,Eight finishes\nFor slopes exceeding 33 degree...,https://modernfan.com/wp-content/uploads/mfc-p...,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2486,2519,2519,https://modernfan.com/ceiling-fans/45-degree-s...,45 Degree Slope Kit,https://modernfan.com/wp-content/uploads/mfc-p...,Eight finishes\nFor slopes exceeding 33 degree...,https://modernfan.com/wp-content/uploads/mfc-p...,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2487,2520,2520,https://modernfan.com/ceiling-fans/45-degree-s...,45 Degree Slope Kit,https://modernfan.com/wp-content/uploads/mfc-p...,Eight finishes\nFor slopes exceeding 33 degree...,https://modernfan.com/wp-content/uploads/mfc-p...,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2488,2521,2521,https://modernfan.com/ceiling-fans/45-degree-s...,45 Degree Slope Kit,https://modernfan.com/wp-content/uploads/mfc-p...,Eight finishes\nFor slopes exceeding 33 degree...,https://modernfan.com/wp-content/uploads/mfc-p...,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [97]:
df.to_excel('moderfan_new.xlsx', index = False)

In [73]:
df.reset_index(inplace = True)

In [70]:
df = df.drop([2155], axis = 0)

In [66]:
df['images'][2155].split('\n')

[nan,
 'https://modernfan.com/wp-content/uploads/2021/03/Gusto_Flush_132B.jpg',
 'https://modernfan.com/wp-content/uploads/2021/03/Gusto_Flush_Gallery-1024x624.jpg']

In [38]:
df['images'][1]

'https://modernfan.com/wp-content/uploads/2020/11/ARB-GW-MP-0574-1024x632.jpg\nhttps://modernfan.com/wp-content/uploads/2020/11/ARB-DB-EB-0698-1024x673.jpg\nhttps://modernfan.com/wp-content/uploads/2020/11/ARB-DB-MP-under-1024x991.jpg\nhttps://modernfan.com/wp-content/uploads/2020/11/ARB-GT-GY-0629-1024x756.jpg\nhttps://modernfan.com/wp-content/uploads/2020/11/ARB-GW-MP-0562-1024x728.jpg\nhttps://modernfan.com/wp-content/uploads/2020/11/ARB-DB-EB-0699-1024x694.jpg\nhttps://modernfan.com/wp-content/uploads/2020/11/ARB-GT-GY-0634-1024x683.jpg'

In [149]:
df = pd.read_excel(r"C:\Users\kchai\Downloads\modernfan.xlsx")
df

,name,price,sku,product_url,hero_image,short description,images,variant_image,option(Fan Finish),option(Light Option),...,categories,available_new,voltage,bulb_type,bulb_text,UL Listed,certifications,warranty & care,position,notes
0,Arbor DC | 54″,$615.00,ARB-DB-54-EB-NL-WC,https://modernfan.com/ceiling-fans/arbor-dc-54/,https://modernfan.com/wp-content/uploads/mfc-p...,Three Fan Finishes\nSolid Wood Blades in Five ...,https://modernfan.com/wp-content/uploads/2020/...,https://modernfan.com/wp-content/uploads/mfc-p...,Dark Bronze: https://modernfan.com/wp-content/...,No Light: https://modernfan.com/wp-content/upl...,...,Ceiling Fans,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Arbor DC | 54″,$615.00,ARB-DB-54-EB-NL-RC,https://modernfan.com/ceiling-fans/arbor-dc-54/,https://modernfan.com/wp-content/uploads/mfc-p...,Three Fan Finishes\nSolid Wood Blades in Five ...,https://modernfan.com/wp-content/uploads/2020/...,https://modernfan.com/wp-content/uploads/mfc-p...,Dark Bronze: https://modernfan.com/wp-content/...,No Light: https://modernfan.com/wp-content/upl...,...,Ceiling Fans,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Arbor DC | 54″,$640.00,ARB-DB-54-EB-NL-CC,https://modernfan.com/ceiling-fans/arbor-dc-54/,https://modernfan.com/wp-content/uploads/mfc-p...,Three Fan Finishes\nSolid Wood Blades in Five ...,https://modernfan.com/wp-content/uploads/2020/...,https://modernfan.com/wp-content/uploads/mfc-p...,Dark Bronze: https://modernfan.com/wp-content/...,No Light: https://modernfan.com/wp-content/upl...,...,Ceiling Fans,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Arbor DC | 54″,$680.00,ARB-DB-54-EB-870-WC,https://modernfan.com/ceiling-fans/arbor-dc-54/,https://modernfan.com/wp-content/uploads/mfc-p...,Three Fan Finishes\nSolid Wood Blades in Five ...,https://modernfan.com/wp-content/uploads/2020/...,https://modernfan.com/wp-content/uploads/mfc-p...,Dark Bronze: https://modernfan.com/wp-content/...,17W LED- #870 (+$65.00): https://modernfan.com...,...,Ceiling Fans,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Arbor DC | 54″,$680.00,ARB-DB-54-EB-870-RC,https://modernfan.com/ceiling-fans/arbor-dc-54/,https://modernfan.com/wp-content/uploads/mfc-p...,Three Fan Finishes\nSolid Wood Blades in Five ...,https://modernfan.com/wp-content/uploads/2020/...,https://modernfan.com/wp-content/uploads/mfc-p...,Dark Bronze: https://modernfan.com/wp-content/...,17W LED- #870 (+$65.00): https://modernfan.com...,...,Ceiling Fans,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2401,Ball Flush | 42″,450,BAL-FM-GW-42-MP-NL-005,https://modernfan.com/ceiling-fans/ball-flush-42/,https://modernfan.com/wp-content/uploads/mfc-p...,Brushed Aluminum or Gloss White\nThree Blade C...,https://modernfan.com/wp-content/uploads/2022/...,https://modernfan.com/wp-content/uploads/mfc-p...,Gloss White: https://modernfan.com/wp-content/...,No Light: https://modernfan.com/wp-content/upl...,...,Ceiling Fans,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2402,Ball Flush | 42″,490,BAL-FM-GW-42-MP-652-002,https://modernfan.com/ceiling-fans/ball-flush-42/,https://modernfan.com/wp-content/uploads/mfc-p...,Brushed Aluminum or Gloss White\nThree Blade C...,https://modernfan.com/wp-content/uploads/2022/...,https://modernfan.com/wp-content/uploads/mfc-p...,Gloss White: https://modernfan.com/wp-content/...,20W LED- #652 (+$80.00): https://modernfan.com...,...,Ceiling Fans,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2403,Ball Flush | 42″,510,BAL-FM-GW-42-MP-652-003,https://modernfan.com/ceiling-fans/ball-flush-42/,https://modernfan.com/wp-content/uploads/mfc-p...,Brushed Aluminum or Gloss White\nThree Blade C...,https://modernfan.com/wp-content/uploads/2022/...,https://modernfan.com/wp-content/uploads/mfc-p...,Gloss White: https://modernfan.com/wp-content/...,20W LED- #652 (+$80.00): https://modernfan.com...,...,Ceiling Fans,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2404,Ball Flush | 42″,510,BAL-FM-GW-42-MP-652-004,https://modernfan.com/ceiling-fans/ball-flush-

In [158]:
for i in range(len(df)):
    df['images'][i] = "".join([s for s in df['images'][i] .strip().splitlines(True) if s.strip()])

C:\Users\kchai\AppData\Local\Temp\ipykernel_9936\1134793504.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['images'][i] = "".join([s for s in df['images'][i] .strip().splitlines(True) if s.strip()])


In [160]:
df.to_excel('modernfan_new.xlsx', index = False)

In [96]:
df[df['variant_image'].isna()]['product_url']

Series([], Name: product_url, dtype: object)